<a href="https://colab.research.google.com/github/apoorvrocks/Machine-Learning-Assignment-1/blob/main/GMM_modified.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



1.   200 classes for training.
2.   Each class has 500 images.
3.   The test set contains 10,000 images.
4.   All images are 64x64x3 colored ones.






In [1]:
from zipfile import ZipFile
import os
import random
import numpy as np
from skimage import color, io
from skimage.transform import resize
import matplotlib.pyplot as plt
import gc
from scipy.stats import multivariate_normal

UNZIP DATA

In [2]:
with ZipFile('/content/drive/MyDrive/IISc/PRNN/tiny-imagenet-200.zip', 'r') as zipObj:
   zipObj.extractall()

SET PATHS

In [3]:
test_path = '/content/tiny-imagenet-200/test'
train_path = '/content/tiny-imagenet-200/train'
validation_path = '/content/tiny-imagenet-200/val'

In [4]:
x = []
NO_CLASSES = 10
NO_IMAGES = 20
class_folders = os.listdir(train_path)

# loop over classes
for class_folder in class_folders[:NO_CLASSES]: 

    image_folder_path = train_path + os.path.sep + class_folder + os.path.sep + 'images' + os.path.sep
    image_file_names = os.listdir(image_folder_path)

    # loop over images
    for image_file_name in image_file_names[:NO_IMAGES]: # loop over all images in a class
        img = io.imread(image_folder_path + image_file_name)
        if len(img.shape) == 3:
            imgGray = np.mean(img, axis=2)
            img_resized = resize(imgGray, (28, 28))
            flat_image = img_resized.flatten() 
            
            x.append(flat_image)



In [5]:
def normalize_matrix(mat):
  for c in range(len(mat[0])):
    col = mat[:,c]
    mn = min(col)
    mx = max(col)
    mat[:,c] = (col - mn)/(mx - mn)

  return mat

In [6]:
# X = normalize_matrix(np.array(x))
# del x
# gc.collect()
# print(X.shape)
X= np.array(x)

In [7]:
X

array([[ 89.71715456,  90.77363769,  92.01004037, ..., 124.28290969,
        110.56004536,  98.95139386],
       [107.22667016, 103.84486798, 106.44781894, ...,  62.41810633,
         41.27900477,  25.02850709],
       [239.        , 239.69277482, 240.46613078, ...,  10.52379452,
         17.19889569,  18.21209894],
       ...,
       [ 40.23302864,  50.96676781, 115.93372679, ..., 105.38647025,
         88.56831407,  85.05038104],
       [241.06315372, 237.25124719, 240.3545158 , ...,  81.52925689,
         72.09343204,  79.3475017 ],
       [129.02399571, 130.35107447,  65.13915677, ..., 106.44872491,
         77.06142671,  53.21392734]])

In [10]:
class GMM:
    def gaussian(self, X, mu, det, inv_cov):
    
        n = self.n_features
        diff = X - mu.T
        a = np.log(det)
        b = n * np.log(2 * np.pi)
        # print('diff', diff.shape)
        # print('cov shape', cov.shape)

        c = np.dot(np.dot(diff, inv_cov), diff.T)
        # print('(',a,b,c,') ', end=' ')
        # print('shapes', diff.shape, a.shape, b.shape, c.shape)
        return np.exp(-0.5*(a+b+c))

    def __init__(self, n_features, n_clusters):

        self.clusters = {}
        self.n_clusters = n_clusters
        self.n_features = n_features

        self.clusters['pi'] =( 1.0 / n_clusters) * np.ones(n_clusters, dtype=np.float64)
        self.clusters['mu'] = np.random.rand(self.n_clusters, self.n_features)
        cov = []

        for i in range(n_clusters):
            cov.append(np.eye(self.n_features)*0.9)
        self.clusters['cov'] = np.array(cov)     
        # self.clusters['cov'] = np.random.randint(1, 2, size=(n_clusters, n_features, n_features))  
        print(self.clusters['pi'].shape)
        print(self.clusters['mu'].shape)
        print(self.clusters['cov'].shape)


    def expectation_step(self, X):

        # calculate responsibility for each data point, for each class
        self.resp = np.empty((self.N, self.n_clusters), dtype=np.float64)
        self.likelihood = 0
        print('E step')

        for k in range(self.n_clusters):
            inv_cov = np.linalg.inv(self.clusters['cov'][k])
            det = np.linalg.det(self.clusters['cov'][k])
            print('det=', det)
            for n, datapoint in enumerate(X):
                # print(f'E step:\tn:{n} k:{k}')
                # prob = self.gaussian(datapoint, self.clusters['mu'][k], det, inv_cov)
                prob = multivariate_normal.pdf(datapoint, mean = self.clusters['mu'][k], cov = self.clusters['cov'][k])
                # print('probs',prob)
                self.resp[n][k] = self.clusters['pi'][k] * prob

            self.likelihood += np.log(self.resp[n][k])

        for n in range(self.N):
            # print('sum(resp)', np.sum(self.resp[n]))
            self.resp[n] = self.resp[n] / np.sum(self.resp[n])




    def maximization_step(self, X):
        print('M step')
        class_resp = np.sum(self.resp, axis=0) # get class responsibility
        print('responsibility ',class_resp)
        for k in range(self.n_clusters):
            
            # find pi for each cluster
            self.clusters['pi'][k] = class_resp[k] / self.N

            # find new mean for each cluster
            mean = np.matmul(self.resp[:, k], X) / class_resp[k]
            self.clusters['mu'][k] = mean


            # find covariance matrix for each cluster
            cov = np.zeros((self.n_features, self.n_features), dtype=np.float64)
            for n in range(self.N):
                v = X[n] - mean
                cov += self.resp[n][k] * np.outer(v, v)
            
            cov /= class_resp[k]
            self.clusters['cov'][k] = cov



    def fit(self, X, n_epochs):

        self.N = X.shape[0]

        for i in range(n_epochs):
        
            self.expectation_step(X)
            self.maximization_step(X)


            print('Epoch: ', i + 1, 'Likelihood: ', self.likelihood)
            
        # for i, cluster in enumerate(clusters):
        #     scores[:, i] = np.log(cluster['gamma_nk']).reshape(-1)
            
        # return clusters, likelihoods, scores, sample_likelihoods



NO_CLUSTERS = 2
NO_EPOCHS = 5

gmm = GMM(X.shape[1], NO_CLUSTERS)
gmm.fit(X, NO_EPOCHS)


(2,)
(2, 784)
(2, 784, 784)
E step
det= 1.3369876648018035e-36


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:53: RuntimeWarning: divide by zero encountered in log


det= 1.3369876648018035e-36


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:57: RuntimeWarning: invalid value encountered in true_divide


M step
responsibility  [nan nan]
Epoch:  1 Likelihood:  -inf
E step
det= nan


/usr/local/lib/python3.7/dist-packages/numpy/linalg/linalg.py:2158: RuntimeWarning: invalid value encountered in det
  r = _umath_linalg.det(a, signature=signature)


ValueError: ignored